<a href="https://colab.research.google.com/github/henriquehsilva/i2a2-challenges/blob/master/I2A2_Bone_Age_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython.display import HTML, display

# I2A2 - Bone Age Regression
---



## Contexto
> ### Criar um modelo para prever a idade óssea de crianças pelo raio-x da mão.


*   O conjunto de dados de treinamento vem da competição RSNA 2017
*   O conjunto de dados de teste é de um hospital brasileiro

**Diretivas primárias:** 


1. Generalizar o modelo para lidar com a mudança de distribuição entre os conjuntos de dados;
2. O conjunto de dados de **TREINAMENTO contém apenas imagens com a mão esquerda**, o conjunto de dados de **TESTE contém algumas imagens com as duas mãos**;
3. Ajustar o modelo, para lidar com a variância de resultados entre pacientes de sexos opostos;

---
## Visão geral

Uma das principais preocupações da medicina e da odontologia legal é a busca
de mecanismos que permitam a determinação da idade de pessoas e, para o desenvolvimento
deste trabalho, escolheu-se a **região de mão e pulso**, para a determinação da idade e do
desenvolvimento ósseo, devido a seqüência cronológica que a mesma apresenta bem como a
quantidade de ossos e epífises em uma área não muito extensa sendo possível realizar uma
única tomada radiográfica, evitando-se exposições desnecessárias ao paciente. 



In [2]:
video_ids = ['Zs1oC_upTvM', 'WIk5S0BHotY']

def embed_video(video_ids):
  videos_str = ""
  
  for video_id in video_ids: 
    videos_str += (
      f"<iframe width='560' height='315' src='https://www.youtube.com/embed/{video_id}'.format(test)" \
       "frameborder='0' allow='accelerometer; autoplay; encrypted-media; gyroscope;" \
       " picture-in-picture' allowfullscreen></iframe>"
      )
    
  return HTML(videos_str)

embed_video(video_ids)

Após estudo da tese de mestrado do cirurgião dentista Mauricio Roberto Bosquiero: [DETERMINAÇÃO DA MATURIDADE ESQUELÉTICA
E ESTIMATIVA DA IDADE ATRA VÊS DE
RADIOGRAFIAS CARPAIS.](https://storage.googleapis.com/bone-age-regression.henriquesilva.dev/docs/tese_mauricio_roberto.pdf), podemos chegar a uma terceira diretiva, referente a clara variância da maturidade esquelética de pacientes de mesma idade e sexos opostos.

---


> ### Pretendo avaliar o desempenho dos seguintes modelos pré-treinados: <br />
**ResNet-50**, **Inception-V3** e **VGG-16**, a fim de criar um desenho apropriado da camada pré-treinada para CNNs na previsão da idade óssea.

In [3]:
# Sweetviz: Python library for Exploratory data analysis (EDA)
!pip install sweetviz

In [87]:
# TensorFlow e tf.keras
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import load_img
print(f"TensorFlow V{tf.__version__} 🦾")

TensorFlow V2.2.0 🦾


In [5]:
# Libraries auxiliares
import pandas as pd
import sweetviz
import requests

from sklearn.model_selection import train_test_split
from google.colab import auth
from datetime import datetime

# Configurar conjuntos de desenvolvimento e de teste
---

O **Google Cloud Storage** foi escolhido como centralizador dos arquivos de suporte, bem como aos novos arquivos e relatórios gerados, por oferecer a disponibilidade e a capacidade necessárias para se trabalhar de forma distribuida.


In [6]:
GCP_STORAGE_BASE_URI = "https://storage.googleapis.com/bone-age-regression.henriquesilva.dev"

TRAIN_URI = f"{GCP_STORAGE_BASE_URI}/train.csv"
TEST_URI = f"{GCP_STORAGE_BASE_URI}/test.csv"

full_train_df = pd.read_csv(TRAIN_URI)
test_df = pd.read_csv(TEST_URI)

# Adicionar séries da categoria da idade óssea e link da imagem do raio-x da mão

In [7]:
full_train_df['boneage_category'] = pd.cut(full_train_df['boneage'], 10)

In [8]:
def image_link_generator(path_uri, file_name):
  image_link = f"{path_uri}/images/{file_name}"
  return image_link

full_train_df['image_link'] = full_train_df['fileName'].map(
  lambda file_name: image_link_generator(GCP_STORAGE_BASE_URI, file_name)
)

In [9]:
HTML(full_train_df.head(5).to_html(render_links=True, escape=False))

,fileName,patientSex,boneage,boneage_category,image_link
0,1377.png,F,180,"(159.9, 182.6]",https://storage.googleapis.com/bone-age-regression.henriquesilva.dev/images/1377.png
1,1378.png,F,12,"(0.773, 23.7]",https://storage.googleapis.com/bone-age-regression.henriquesilva.dev/images/1378.png
2,1379.png,F,94,"(91.8, 114.5]",https://storage.googleapis.com/bone-age-regression.henriquesilva.dev/images/1379.png
3,1380.png,M,120,"(114.5, 137.2]",https://storage.googleapis.com/bone-age-regression.henriquesilva.dev/images/1380.png
4,1381.png,F,82,"(69.1, 91.8]",https://storage.googleapis.com/bone-age-regression.henriquesilva.dev/images/1381.png


# Dividir dados em treinamento e validação

In [10]:
train_df, valid_df = train_test_split(
  full_train_df, 
  test_size = 0.25, 
  random_state = 2018,
  stratify = full_train_df['boneage_category']
)

print('train', train_df.shape[0], 'validation', valid_df.shape[0])

train 9458 validation 3153


# Análise exploratória primária de dados de desenvolvimento


In [11]:
columns = ['fileName', 'boneage_category', 'image_link']
train = train_df.drop(columns, axis=1)
valid = valid_df.drop(columns, axis=1)

A execução deste comando executará a análise e criará o objeto de relatório. Para obter a saída, basta usar o comando `show_html()`:

In [12]:
data_report = sweetviz.compare([train, "Train"], [valid, "Valid"], "boneage")

:FEATURES DONE:                    |█████████████████████| [100%]   00:01  -> (00:00 left)
:PAIRWISE DONE:                    |█████████████████████| [100%]   00:00  -> (00:00 left)


Creating Associations graph... DONE!


In [13]:
now = datetime.now()
timestamp = now.strftime("%Y%m%d%H%M%S")

report_file = f"{timestamp}_report_view.html"

data_report.show_html(report_file)

# Exportar relatório

In [14]:
auth.authenticate_user()

GCP_PROJECT_ID = 'i2a2-283020'
GCP_BUCKET_NAME = 'bone-age-regression.henriquesilva.dev'

!gcloud config set project {GCP_PROJECT_ID}

!gsutil cp /content/"{report_file}" gs://{GCP_BUCKET_NAME}/reports/


Updated property [core/project].
Copying file:///content/20200715182212_report_view.html [Content-Type=text/html]...
-
Operation completed over 1 objects/553.7 KiB.                                    


In [15]:
response = requests.get(f"{GCP_STORAGE_BASE_URI}/reports/{report_file}")

if (response.status_code == 200):
  print(f"Open your report it using the default browser like link: {response.url}")
else:
  print("Sorry, the report not found.")

Open your report it using the default browser like link: https://storage.googleapis.com/bone-age-regression.henriquesilva.dev/reports/20200715182212_report_view.html


# Separar os dados de desenvolvimento quanto ao sexo da paciente

## train

In [60]:
female_train_df = train_df[(train_df.patientSex == 'F')]
male_train_df = train_df[(train_df.patientSex == 'M')]

## valid

In [62]:
female_valid_df = valid_df[(valid_df.patientSex == 'F')]
male_valid_df = valid_df[(valid_df.patientSex == 'M')]

In [4]:
# import pathlib

# dataset_url = "https://storage.googleapis.com/bone-age-regression.henriquesilva.dev/images"

# data_dir = tf.keras.utils.get_file(origin=dataset_url, 
#                                    fname='images', 
#                                    untar=True)

# data_dir = pathlib.Path(data_dir)

!gsutil cp --help

NAME
  cp - Copy files and objects


SYNOPSIS

  gsutil cp [OPTION]... src_url dst_url
  gsutil cp [OPTION]... src_url... dst_url
  gsutil cp [OPTION]... -I dst_url



DESCRIPTION
  The gsutil cp command allows you to copy data between your local file
  system and the cloud, copy data within the cloud, and copy data between
  cloud storage providers. For example, to upload all text files from the
  local directory to a bucket you could do:

    gsutil cp *.txt gs://my-bucket

  Similarly, you can download text files from a bucket by doing:

    gsutil cp gs://my-bucket/*.txt .

  If you want to copy an entire directory tree you need to use the -r option.
  For example, to upload the directory tree "dir":

    gsutil cp -r dir gs://my-bucket

  If you have a large number of files to transfer you might want to use the
  top-level gsutil -m option (see "gsutil help options"), to perform a
  parallel (multi-threaded/multi-processing) copy:

    gsutil -m cp -r dir gs://my-bucket

  You can

In [68]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
IMG_SIZE = (384, 384) # slightly smaller than vgg16 normally expects
core_idg = ImageDataGenerator(samplewise_center=False, 
                              samplewise_std_normalization=False, 
                              horizontal_flip = True, 
                              vertical_flip = False, 
                              height_shift_range = 0.15, 
                              width_shift_range = 0.15, 
                              rotation_range = 5, 
                              shear_range = 0.01,
                              fill_mode = 'nearest',
                              zoom_range=0.25,
                             preprocessing_function = preprocess_input)

Using TensorFlow backend.


In [77]:
import os

In [80]:
def flow_from_dataframe(img_data_gen, in_df, path_col, y_col, **dflow_args):
    # base_dir = in_df[path_col].value
    print('## Ignore next message from keras, values are replaced anyways')
    df_gen = img_data_gen.flow_from_directory(base_dir, 
                                     class_mode = 'sparse',
                                    **dflow_args)
    df_gen.filenames = in_df[path_col].values
    df_gen.classes = np.stack(in_df[y_col].values)
    df_gen.samples = in_df.shape[0]
    df_gen.n = in_df.shape[0]
    df_gen._set_index_array()
    df_gen.directory = '' # since we have the full path
    print('Reinserting dataframe: {} images'.format(in_df.shape[0]))
    return df_gen

In [74]:
!gsutil ls gs://{GCP_BUCKET_NAME}/reports/

gs://bone-age-regression.henriquesilva.dev/reports/
gs://bone-age-regression.henriquesilva.dev/reports/20200714002253_report_view.html
gs://bone-age-regression.henriquesilva.dev/reports/20200715013044_report_view.html
gs://bone-age-regression.henriquesilva.dev/reports/20200715013336_report_view.html
gs://bone-age-regression.henriquesilva.dev/reports/20200715180959_report_view.html
gs://bone-age-regression.henriquesilva.dev/reports/20200715182212_report_view.html


In [81]:
train_gen = flow_from_dataframe(core_idg, train_df, 
                             path_col = 'image_link',
                            y_col = 'boneage_zscore', 
                            target_size = IMG_SIZE,
                             color_mode = 'rgb',
                            batch_size = 32)

valid_gen = flow_from_dataframe(core_idg, valid_df, 
                             path_col = 'image_link',
                            y_col = 'boneage_zscore', 
                            target_size = IMG_SIZE,
                             color_mode = 'rgb',
                            batch_size = 256) # we can use much larger batches for evaluation
# used a fixed dataset for evaluating the algorithm
test_X, test_Y = next(flow_from_dataframe(core_idg, 
                               valid_df, 
                             path_col = 'image_link',
                            y_col = 'boneage_zscore', 
                            target_size = IMG_SIZE,
                             color_mode = 'rgb',
                            batch_size = 1024)) # one big batch

## Ignore next message from keras, values are replaced anyways


FileNotFoundError: ignored